In [ ]:
import dspy
from dotenv import load_dotenv
from phoenix.otel import register

load_dotenv()

tracer_provider = register(
  project_name="dspy_simpler",
  auto_instrument=True
)

lm = dspy.LM('azure/gpt-4.1-nano', cache=True)
dspy.configure(lm=lm)

def evaluate_math(expression: str):
    return dspy.PythonInterpreter({}).execute(expression)

def search_wikipedia(query: str):
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=3)
    return [x['text'] for x in results]

react = dspy.ReAct("question -> answer: float", tools=[evaluate_math, search_wikipedia])

pred = react(question="What is 9362158 divided by the year of birth of David Gregory of Kinnairdy castle?")
print(pred.answer)

Failed to export batch code: 404, reason: <!doctype html><meta charset="utf-8"><meta name=viewport content="width=device-width, initial-scale=1"><title>404</title>404 Not Found
Failed to export batch code: 404, reason: <!doctype html><meta charset="utf-8"><meta name=viewport content="width=device-width, initial-scale=1"><title>404</title>404 Not Found
Failed to export batch code: 404, reason: <!doctype html><meta charset="utf-8"><meta name=viewport content="width=device-width, initial-scale=1"><title>404</title>404 Not Found
Failed to export batch code: 404, reason: <!doctype html><meta charset="utf-8"><meta name=viewport content="width=device-width, initial-scale=1"><title>404</title>404 Not Found
Failed to export batch code: 404, reason: <!doctype html><meta charset="utf-8"><meta name=viewport content="width=device-width, initial-scale=1"><title>404</title>404 Not Found
Failed to export batch code: 404, reason: <!doctype html><meta charset="utf-8"><meta name=viewport content="width=d

5761.33


In [7]:
import pandas as pd
from IPython.display import display, HTML

def parse_lm_history(history):
    structured_data = []
    
    for i, interaction in enumerate(history):
        # Extract messages
        messages = interaction.get('messages', [])
        user_message = next((msg['content'] for msg in messages if msg.get('role') == 'user'), '')
        
        # Extract output
        output = interaction.get('outputs', [''])[0]
        
        # Get model name and timestamp
        model = interaction.get('model', '')
        timestamp = interaction.get('timestamp', '')
        
        # Calculate tokens and cost if available
        usage = interaction.get('usage', {})
        tokens = usage.get('total_tokens', 'N/A')
        cost = interaction.get('cost', 'N/A')
        
        structured_data.append({
            'Interaction': i+1,
            'Model': model,
            'Timestamp': timestamp,
            'User Message': user_message[:100] + '...' if len(user_message) > 100 else user_message,
            'Output': output[:100] + '...' if len(output) > 100 else output,
            'Tokens': tokens,
            'Cost': cost
        })
    
    return pd.DataFrame(structured_data)

# Create and display a formatted DataFrame
history_df = parse_lm_history(lm.history)
display(history_df)

# Function to display detailed interaction
def show_detailed_interaction(interaction_idx):
    if 0 <= interaction_idx < len(lm.history):
        interaction = lm.history[interaction_idx]
        
        print(f"Detailed Interaction #{interaction_idx+1}")
        print("="*50)
        
        # Display messages in conversation format
        print("Conversation:")
        for msg in interaction.get('messages', []):
            role = msg.get('role', 'unknown')
            content = msg.get('content', '')
            print(f"\n{role.upper()}:\n{content}")
        
        print("\n" + "="*50)
        print("OUTPUT:")
        print(interaction.get('outputs', ['N/A'])[0])
        
        print("\n" + "="*50)
        print("METADATA:")
        for key, value in interaction.items():
            if key not in ['messages', 'outputs', 'response']:
                print(f"{key}: {value}")
    else:
        print(f"Interaction {interaction_idx+1} not found in history.")

# Show example usage - uncomment and modify index to see details
print("\nTo see detailed information for a specific interaction, run:")
print("show_detailed_interaction(0)  # Replace 0 with interaction index")

,Interaction,Model,Timestamp,User Message,Output,Tokens,Cost
0,1,azure/gpt-4.1-nano,2025-05-13T15:25:36.199933,[[ ## question ## ]]\nWhat is 9362158 divided ...,[[ ## next_thought ## ]]\nI need to find the y...,N/A,0.000094
1,2,azure/gpt-4.1-nano,2025-05-13T15:25:37.736800,[[ ## question ## ]]\nWhat is 9362158 divided ...,[[ ## next_thought ## ]]\nThe search results i...,N/A,0.000127
2,3,azure/gpt-4.1-nano,2025-05-13T15:25:41.532403,[[ ## question ## ]]\nWhat is 9362158 divided ...,[[ ## next_thought ## ]]\nThe division of 9362...,N/A,0.000131
3,4,azure/gpt-4.1-nano,2025-05-13T15:25:42.706183,[[ ## question ## ]]\nWhat is 9362158 divided ...,[[ ## reasoning ## ]]\nThe question asks for t...,N/A,0.000109



To see detailed information for a specific interaction, run:
show_detailed_interaction(0)  # Replace 0 with interaction index


In [12]:
for idx in range(len(lm.history)):
  show_detailed_interaction(idx)

Detailed Interaction #1
Conversation:

SYSTEM:
Your input fields are:
1. `question` (str)
2. `trajectory` (str)
Your output fields are:
1. `next_thought` (str)
2. `next_tool_name` (Literal['evaluate_math', 'search_wikipedia', 'finish'])
3. `next_tool_args` (dict[str, Any])
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## trajectory ## ]]
{trajectory}

[[ ## next_thought ## ]]
{next_thought}

[[ ## next_tool_name ## ]]
{next_tool_name}        # note: the value you produce must exactly match (no extra characters) one of: evaluate_math; search_wikipedia; finish

[[ ## next_tool_args ## ]]
{next_tool_args}        # note: the value you produce must adhere to the JSON schema: {"type": "object", "additionalProperties": true}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `question`, produce the fields `answer`.
        
        You are an Agent. In each e